# Web scraping

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep

def obtener_datos_libro(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extraer título
    title_element = soup.select_one("#title h1")
    title = title_element.text.strip() if title_element else None

    # Extraer autores
    author_elements = soup.select("#autor a.dinSource")
    authors = ", ".join([author.text.strip() for author in author_elements])

    # Extraer géneros (si hay múltiples géneros, concatenar)
    genre_elements = soup.select("#genero a.dinSource")
    genres = ", ".join([genre.text.strip() for genre in genre_elements])

    # Extraer sinopsis
    synopsis_element = soup.select_one("#sinopsis span")
    synopsis = synopsis_element.text.strip() if synopsis_element else None

    return {
        "Title": title,
        "Authors": authors,
        "Genres": genres,
        "Synopsis": synopsis
    }

url_base = "https://ww3.lectulandia.com/"
response = requests.get(url_base)
soup = BeautifulSoup(response.content, 'html.parser')

genero_links = soup.select("a[href*='/genero/']")
genero_urls = [link.get('href') for link in genero_links if link.get('href') and link.get('href') != 'javascript:;']

books_data = []
min_books_per_genre = 10
books_per_genre = {}  # Diccionario para mantener un registro de cuántos libros se han recopilado por género

for genre_url in genero_urls:
    genre_name = genre_url.split("/")[-2]
    full_genre_url = f"https://ww3.lectulandia.com{genre_url}"

    page_number = 1
    while books_per_genre.get(genre_name, 0) < min_books_per_genre:
        response = requests.get(f"{full_genre_url}/page/{page_number}/")
        soup = BeautifulSoup(response.content, 'html.parser')

        book_links = soup.select("a.card-click-target[tabindex='-1']")
        book_urls = [link.get('href') for link in book_links if link.get('href') and link.get('href') != 'javascript:;']

        if not book_urls:
            break  # Rompe el bucle si no se encuentran más libros en la página actual

        for book_url in book_urls:
            full_book_url = f"https://ww3.lectulandia.com{book_url}"
            book_data = obtener_datos_libro(full_book_url)
            if book_data["Title"]:
                book_data["Genres"] = book_data.get("Genres", "") + ", " + genre_name if book_data.get("Genres") else genre_name
                books_data.append(book_data)

                # Actualizar el contador de libros por género
                books_per_genre[genre_name] = books_per_genre.get(genre_name, 0) + 1

            if books_per_genre[genre_name] >= min_books_per_genre:
                break

        page_number += 1  # Incrementar el número de página para la siguiente iteración

# Convertir los datos de los libros a un DataFrame de Pandas
books_df = pd.DataFrame(books_data)

# Asegurarse de que no haya duplicados en las columnas de género
if 'Genre' in books_df.columns:
    books_df.drop(columns=['Genre'], inplace=True)

print(books_df.head())

# Guardar el DataFrame en un archivo CSV
books_df.to_csv("books_data.csv", index=False)

# Mostrar cuántos libros se recopilaron por género
for genre, count in books_per_genre.items():
    print(f"{genre}: {count}")

                                               Title  \
0                         El mundo de la arqueología   
1           Atapuerca: 40 años inmersos en el pasado   
2        La vuelta al mundo en seis millones de años   
3  Evolución humana. Prehistoria y origen de la c...   
4                                       Epigramas II   

                             Authors  \
0                        C. W. Ceram   
1  Eudald Carbonell, Rosa M. Tristan   
2   Andrea Brunelli, Guido Barbujani   
3                       Roberto Sáez   
4              Marco Valerio Marcial   

                                              Genres  \
0              Arqueología, Divulgación, arqueologia   
1  Arqueología, Ciencias naturales, Divulgación, ...   
2  Arqueología, Ciencias naturales, Divulgación, ...   
3  Arqueología, Ciencias naturales, Divulgación, ...   
4  Arqueología, Humor, Otros, Poesía, Referencia,...   

                                            Synopsis  
0  Las maravillas de la tumba 

In [ ]:
import pandas as pd
df = pd.read_csv("books_data.csv")
df.tail()

,Title,Authors,Genres,Synopsis
938,Viaje por mar con Don Quijote,Thomas Mann,"Crítica y teoría literaria, Crónica, Viajes, v...",Thomas Mann viaja por primera vez a Estados Un...
939,Los exploradores de la reina,César Vidal,"Biografía, Divulgación, Historia, Viajes, viajes","En la época de la reina Victoria (1819-1901), ..."
940,Diario de un naturalista alrededor del mundo (...,Charles Darwin,"Crónica, Viajes, viajes",A los 22 años Charles Darwin tuvo la oportunid...
941,La patria capicúa,Martín Caparrós,"Crónica, Viajes, viajes",Si alguna vez me ha dado mucha envidia un bigo...
942,Himalaya,Erika Fatland,"Crónica, Historia, Política, Viajes, viajes",Erika Fatland nos conduce en esta obra a las a...


In [ ]:
# Cargar el CSV en un DataFrame
books_df = pd.read_csv("books_data.csv")

# Eliminar las filas con reseñas vacías del DataFrame original
books_df.dropna(subset=['Synopsis'], inplace=True)
books_df = books_df[books_df['Synopsis'].str.strip() != '']

# Resetear los índices del DataFrame
books_df.reset_index(drop=True, inplace=True)

# Guardar el DataFrame modificado en el mismo archivo CSV
books_df.to_csv("books_data.csv", index=False)

print(f"Cantidad de libros después de dropear los vacíos: {books_df.shape[0]}")

Cantidad de libros después de dropear los vacíos: 780


In [ ]:
import pandas as pd

# Cargar el CSV en un DataFrame
books_df = pd.read_csv('books_data.csv')

# Verificar duplicados en base a una combinación de columnas relevantes (por ejemplo, 'Title', 'Authors')
duplicados = books_df.duplicated(subset=['Title', 'Authors'])

# Contar el número de duplicados
num_duplicados = duplicados.sum()
print(f"Hay {num_duplicados} libros duplicados en el DataFrame.")

# Mostrar las filas duplicadas
libros_duplicados = books_df[duplicados]
print(libros_duplicados)

# Eliminar los duplicados y mantener solo una aparición de cada combinación de 'Title' y 'Authors'
books_df.drop_duplicates(subset=['Title', 'Authors'], inplace=True)

# Verificar que se eliminaron los duplicados
num_duplicados_eliminados = books_df.duplicated(subset=['Title', 'Authors']).sum()
print(f"Después de eliminar duplicados, hay {num_duplicados_eliminados} duplicados en el DataFrame.")

# Mostrar el DataFrame sin duplicados
print(books_df)

Hay 152 libros duplicados en el DataFrame.
                             Title                      Authors  \
100                El río del Edén              Richard Dawkins   
102             La realidad oculta                 Brian Greene   
103   Los problemas de la biología           John Maynard Smith   
106     Genes, cerebros y símbolos                 Jordi Agustí   
107      Primates al este del Edén  Juan Ignacio Perez Iglesias   
..                             ...                          ...   
770                  Marca de agua               Joseph Brodsky   
771          En el reino del hielo                Hampton Sides   
772                      Galápagos            Francisco Coloane   
773       El mundo inconmensurable               William Atkins   
775  Viaje por mar con Don Quijote                  Thomas Mann   

                                                Genres  \
100  Biología, Ciencias naturales, Divulgación, cie...   
102  Ciencias exactas, Ciencias natu

# Embedding

In [ ]:
pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 1.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Cargar un modelo preentrenado, por ejemplo, 'distiluse-base-multilingual-cased'
model = SentenceTransformer('distiluse-base-multilingual-cased')

# Crear embeddings para las sinopsis
def get_embedding(text, model):
    return model.encode([text])[0]

# Aplicar la función get_embedding a cada sinopsis y crear una nueva columna en el DataFrame
books_df['Synopsis_Embedding'] = books_df['Synopsis'].apply(lambda x: get_embedding(x, model))

# Guardar el DataFrame modificado en el mismo archivo CSV
books_df.to_csv("books_data.csv", index=False)

print(books_df['Synopsis_Embedding'])

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.39k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

rust_model.ot:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

0      [-0.0100895185, -0.005474609, -0.062393066, -0...
1      [-0.012944715, -0.038699258, -0.04425188, -0.0...
2      [-0.035172235, -0.037026435, 0.061960235, -0.0...
3      [-0.046657335, -0.0003825703, -0.047442634, -0...
4      [-0.046657335, -0.0003825703, -0.047442634, -0...
                             ...                        
774    [0.014810799, -0.0073203393, -0.08192041, -0.0...
776    [0.04040541, -0.036644794, 0.0049037198, -0.00...
777    [0.06964158, -0.007986101, 0.025282016, -0.006...
778    [-0.04085651, -0.0008058777, 0.032985188, -0.0...
779    [-0.055808064, -0.039139464, -0.022852205, -0....
Name: Synopsis_Embedding, Length: 628, dtype: object


# Menú

In [ ]:
pip install python-Levenshtein


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 15.0 MB/s eta 0:00:00


In [ ]:
pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 1.7 MB/s eta 0:00:00


In [22]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from Levenshtein import distance
from unidecode import unidecode

# Cargar el DataFrame con los datos de los libros
books_df = pd.read_csv("books_data.csv")

# Cargar el modelo SentenceTransformer preentrenado
model = SentenceTransformer('distiluse-base-multilingual-cased')

def normalize_text(text):
    return unidecode(text.lower().strip())

def recomendacion_directa():
    print("¿Qué tienes ganas de leer hoy?")
    tema = normalize_text(input("> "))

    # Calcular similitud coseno entre la entrada del usuario y las sinopsis
    embeddings = model.encode(books_df['Synopsis'].tolist())
    user_embedding = model.encode([tema])[0]
    similarities = cosine_similarity([user_embedding], embeddings)[0]

    # Obtener los índices de los libros más similares
    top_indices = similarities.argsort()[-3:][::-1]

    # Mostrar la lista de libros recomendados
    for idx in top_indices:
        print(f"Libro: {books_df.loc[idx, 'Title']}")
        print(f"Autor: {books_df.loc[idx, 'Authors']}")
        print(f"Género: {books_df.loc[idx, 'Genres']}")
        print(f"Sinopsis: {books_df.loc[idx, 'Synopsis']}\n")

def eleccion_por_autor():
    print("¿Qué autor te interesa?")
    autor = normalize_text(input("> "))
    umbral_levenshtein = 3

    # Calcular la distancia de Levenshtein entre el autor ingresado y los autores de los libros
    books_df['Author_Distance'] = books_df['Authors'].apply(lambda x: distance(autor, normalize_text(x)))

    # Filtrar libros cuyo autor tenga una distancia de Levenshtein menor al umbral
    libros_autor = books_df[books_df['Author_Distance'] <= umbral_levenshtein].sort_values(by='Author_Distance')

    if libros_autor.empty:
        print("No se encontraron autores que coincidan con tu búsqueda.")
    else:
        # Mostrar hasta tres libros recomendados
        for _, row in libros_autor.head(3).iterrows():
            print(f"Libro: {row['Title']}")
            print(f"Autor: {row['Authors']}")
            print(f"Género: {row['Genres']}")
            print(f"Sinopsis: {row['Synopsis']}\n")

def eleccion_por_genero_literario():
    print("¿Qué género literario te interesa?")
    genero = normalize_text(input("> "))
    umbral_levenshtein = 3

    # Calcular la distancia de Levenshtein entre el género ingresado y los géneros de los libros
    books_df['Genre_Distance'] = books_df['Genres'].apply(lambda x: min([distance(genero, normalize_text(g)) for g in x.split(',')]))

    # Filtrar libros cuyo género tenga una distancia de Levenshtein menor al umbral
    libros_genero = books_df[books_df['Genre_Distance'] <= umbral_levenshtein].sort_values(by='Genre_Distance').head(3)

    if libros_genero.empty:
        print("No se encontraron géneros que coincidan con tu búsqueda.")
    else:
        # Mostrar los libros recomendados
        for _, row in libros_genero.iterrows():
            print(f"Libro: {row['Title']}")
            print(f"Autor: {row['Authors']}")
            print(f"Género: {row['Genres']}")
            print(f"Sinopsis: {row['Synopsis']}\n")

# Función principal del programa
def main():
    while True:
        print("\n--- Menú de Recomendaciones ---")
        print("1. Recomendación Directa")
        print("2. Elección por Autor")
        print("3. Elección por Género Literario")
        print("4. Salir")

        opcion = input("Seleccione una opción: ")

        if opcion == "1":
            recomendacion_directa()
        elif opcion == "2":
            eleccion_por_autor()
        elif opcion == "3":
            eleccion_por_genero_literario()
        elif opcion == "4":
            print("¡Hasta luego!")
            break
        else:
            print("Opción inválida. Por favor, seleccione una opción válida.")

if __name__ == "__main__":
    main()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(



--- Menú de Recomendaciones ---
1. Recomendación Directa
2. Elección por Autor
3. Elección por Género Literario
4. Salir
Seleccione una opción: 2
¿Qué autor te interesa?
> quino
Libro: Mafalda: Femenino singular
Autor: Quino
Género: Cómic, Filosófico, Humor, Sátira, comic
Sinopsis: Por primera vez recopiladas en un libro, todas las tiras «feministas» de Mafalda.Mafalda, la irreverente niña que ha deleitado a generaciones con su visión humorística del mundo en que vivimos, es una de las más ilustres feministas de nuestra época. Cincuenta años después de su nacimiento, cuando movimientos como Time's Up o #MeToo han dado eco a mujeres de todo el mundo y la lucha por los derechos de las mujeres está más que nunca en el candelero, su lectura del mundo sigue en plena vigencia. Las viñetas del genial Quino adquieren hoy una fuerza extraordinaria y nos ayudan a tomar conciencia del camino recorrido y por recorrer para conseguir la igualdad de género. Las viñetas recogidas en este volumen dan 